# Langchain에서 Neo4j 활용하기
---
- 작성일 : 2024-02-13

In [1]:
import os
import dotenv
dotenv.load_dotenv()

# 코랩 환경에서 실행하고 있다면 os.getenv("OPENAI_API_KEY") 를 지우고 본인의 api key를 입력하면 된다  
API_KEY = os.getenv("OPENAI_API_KEY") # .env 파일에 있는 OPENAI_API_KEY를 가져온다
#NEO4J = os.getenv("NEO4J") # .env 파일에 있는 NEO4J를 가져온다

from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

## 1. Neo4j 설정

### Neo4j 연결하기

In [3]:
graph = Neo4jGraph(
    url="bolt://localhost:7687", username="neo4j", password="neo4j"
)

ValueError: Could not connect to Neo4j database. Please ensure that the username and password are correct

### Neo4j 데이터베이스 생성

In [ ]:
graph.query(
    """
MERGE (m:Movie {name:"Top Gun"})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

In [ ]:
# 그래프에서 변경사항 있을 시 리프레시
graph.refresh_schema()
print(graph.schema) # 결과 확인

## 2. Neo4j DB를 기반으로 쿼리하기

In [ ]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [ ]:
chain.run("Who played in Top Gun?")

### 결과 개수 제한하기

In [ ]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, top_k=2
)
chain.run("Who played in Top Gun?")

### 중간 과정 확인하기 (intermediate step)

In [ ]:
result = chain("Who played in Top Gun?")
print(f"Intermediate steps: {result['intermediate_steps']}")

### 문장이 아닌 그래프 요소를 리턴하기

In [ ]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_direct=True
)
chain.run("Who played in Top Gun?")

## 3. Cyper generation prompt
cypher 생성에 대한 예시를 추가할 수 있다.

In [ ]:
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{title:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [ ]:
chain.run("How many people played in Top Gun?")

### (+)QA에 활용되는 LLM과 Cypher generation에 사용되는 LLM을 다르게 설정하기

In [ ]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    verbose=True,
)

chain.run("Who played in Top Gun?")

### 특정 노드나 관계 타입을 무시하기

In [ ]:
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    verbose=True,
    exclude_types=["Movie"], # 제외할 타입을 명세할 수 있다
)

# Inspect graph schema
print(chain.graph_schema)

### 생성된 cypher에서 relation direction을 확인하는 방법

In [ ]:
chain = GraphCypherQAChain.from_llm(
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    graph=graph,
    verbose=True,
    validate_cypher=True,
)

chain.run("Who played in Top Gun?")